In [1]:
import pyodbc

server = 'localhost'  # Example: 'localhost\SQLEXPRESS' or '192.168.1.10'
database = 'DataWarehouse'
username = 'Konstantin Milonas'
password = os.getenv('SQL_PASSWORD')
driver = '{ODBC Driver 17 for SQL Server}'  # Ensure correct driver version

conn = pyodbc.connect(
    f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}'
)


print("Connected to SQL Server successfully!")
conn.close()

try:
    conn = pyodbc.connect(
        f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}'
    )
    cursor = conn.cursor()
    print("Connected to SQL Server successfully!")

    # Example Query
    cursor.execute("SELECT TOP 5 * FROM your_table_name")
    for row in cursor.fetchall():
        print(row)

    # Close connection
    cursor.close()
    conn.close()
except Exception as e:
    print("Error while connecting to SQL Server:", e)


NameError: name 'os' is not defined